In [ ]:
import pandas as pd 
import folium
from folium import plugins
from shapely import wkt
from shapely.geometry import Point
from shapely.wkt import loads

In [ ]:
tour_df = pd.read_csv('food_related_tour.csv', index_col=0)
tripleg = pd.read_csv('triplegs.csv')
staypoints = pd.read_csv('staypoints.csv')
home = pd.read_csv('home_location.csv', index_col=0)
original_data = pd.read_csv('data.csv')

In [ ]:
tour_df['trip_start_timestamp'] = pd.to_datetime(tour_df['trip_start_timestamp'])
tour_df['trip_end_timestamp'] = pd.to_datetime(tour_df['trip_end_timestamp'])
staypoints['started_at'] = pd.to_datetime(staypoints['started_at'])
staypoints['finished_at'] = pd.to_datetime(staypoints['finished_at'])
tripleg['started_at'] = pd.to_datetime(tripleg['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])
original_data['time'] = pd.to_datetime(original_data['time'])
home = home.dropna(subset=['LAT'])

In [ ]:
#device_group = tour_df.groupby('deviceID').agg({'trip_start_timestamp': 'min', 'trip_end_timestamp': 'max'})

# Then, we filter the rows where the difference between the timestamps is greater than or equal to 60 days
# filtered_device_group = device_group[
#     (device_group['trip_end_timestamp'] - device_group['trip_start_timestamp']) >= datetime.timedelta(days=30)
# ]
#tour_df[tour_df['deviceID'] == 205]

<h3>plot all trips of an indivisual</h3>

In [63]:
index = 205
save = False
# file_out = 'FL_Jacksonville.nosync/Phone/flaged_data/virtual_example/new'
file_out = 'Visualization'

In [64]:
user_tripleg = tripleg[tripleg['user_id'] == index]
user_tour = tour_df[tour_df['deviceID'] == index]
GPS_trace = original_data[original_data['user_id'] == index]
    # Create a map centered at the average of the stop points
my_map = folium.Map(location=[30.35, -81.55], zoom_start=5)

tour_group = None

def add_arrow_polyline(group, arrow_coords, arrow_unicode):
    arrow_polyline = folium.PolyLine(arrow_coords, color="black", weight=2.5, opacity=0.8).add_to(group)
    folium.plugins.PolyLineTextPath(
        arrow_polyline,
        arrow_unicode,
        repeat=True,
        offset=5,
        attributes={'font-weight': 'bold', 'font-size': '15', 'color': 'black'}
    ).add_to(group)
def plot_trip(trip):
    linestring = [(lat, lon) for lon, lat in list(loads(trip['geom']).coords)]
    curr_trip_trace = GPS_trace[(GPS_trace['time'] >= trip['started_at'].tz_localize(None)) & (GPS_trace['time'] < trip['finished_at'].tz_localize(None))]
    # Initialize the first point as the start point
    previous_point = linestring[0]
    for i in range(len(linestring)):
        current_point = linestring[i]
        # Add markers for the start and end points
        if i == 0:
            popup = folium.Popup('Trip Start' + '<br>Time: ' + str(trip['started_at'])[:-6], max_width=200)
            folium.Marker(current_point, popup=popup, icon=folium.Icon(color="blue")).add_to(tour_group)
        elif i == len(linestring) - 1:
            popup = folium.Popup('Trip End' + '<br>Time: ' + str(trip['finished_at'])[:-6], max_width=200)
            folium.Marker(current_point, popup=popup, icon=folium.Icon(color="green")).add_to(tour_group)
        else:
            popup = folium.Popup("Index: " + str(trip.index) + '<br>Trip ID: ' + str(trip['id']) + '<br>Time: ' + str(curr_trip_trace.iloc[i]['time']) + '<br>Point: ' + str(current_point), max_width=200)
            folium.Marker(current_point, popup=popup, icon=folium.Icon(color="black")).add_to(tour_group)
        
        # Add arrow between two points to represent the direction
        arrow_coords = [previous_point, current_point]
        add_arrow_polyline(tour_group, arrow_coords, '\u25BA')
        
        # Update the previous point for the next iteration
        previous_point = current_point
        
    folium.PolyLine(linestring, color="black", weight=2.5, opacity=0.8).add_to(tour_group)
    return ['t', previous_point, linestring[0]] #return trip start, trip end
def plot_stop(stop):
    p = loads(stop['geom'])
    popup = folium.Popup('Stop Point: ' + str(stop['id']) + '<br>Start Time: ' + str(stop['started_at'])[:-6] + '<br>End Time: ' + str(stop['finished_at'])[:-6], max_width=200)
    folium.Marker((p.y, p.x), popup=popup, icon=folium.Icon(color="red")).add_to(tour_group)
    return ['s', (p.y, p.x)]

for curr_tour in user_tour.itertuples():
    tour_group = folium.FeatureGroup(name=f'Tour {curr_tour[0]} (DeviceID: {curr_tour[1]})', show=False)
    
    tpid = eval(curr_tour[2])
    spid = eval(curr_tour[7])

    tps = user_tripleg[user_tripleg['id'].isin(tpid)]
    sps = staypoints[staypoints['id'].isin(spid)]
    tps = tps.sort_values(by='started_at')
    sps = sps.sort_values(by='started_at')
    
    t = 0
    s = 0
    previous_info = []
    
    while t < len(tps) or s < len(sps):
        current_info = []
        if t < len(tps) and s < len(sps):
            if tps.iloc[t]['started_at'] < sps.iloc[s]['started_at']:
                current_info = plot_trip(tps.iloc[t])
                t += 1
            else:
                current_info = plot_stop(sps.iloc[s])
                s += 1
        elif t >= len(tps) and s < len(sps):
            current_info = plot_stop(sps.iloc[s])
            s += 1
        elif s >= len(sps) and t < len(tps):
            current_info = plot_trip(tps.iloc[t])
            t += 1
            
        if len(previous_info) > 0 and len(current_info) > 0:
            arrow_coords = None
            if previous_info[0] == 's' and current_info[0] == 't':
                arrow_coords = [previous_info[1], current_info[2]]
            else:
                arrow_coords = [previous_info[1], current_info[1]]
            add_arrow_polyline(tour_group, arrow_coords, '\u002f')
            
        previous_info = current_info
        
    tour_group.add_to(my_map)

folium.LayerControl().add_to(my_map)

# Add home
user_home = home[home['ID'] == index]
if len(user_home > 0):
    l = (user_home.iloc[0]['LAT-4326'], user_home.iloc[0]['LON-4326'])
    folium.Marker(l, popup="Home" + "<br>Location: " + str(l), icon=folium.Icon(color="orange")).add_to(my_map)

if save == True: my_map.save(file_out + '/all_trip' + str(index) + '.html')
my_map
                